In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns


class OpinionDistribution(object):
    def __init__(self, 
                 trends: dict, 
                 p_o,
                 p_p,
                 iteration: int | str = -1, 
                 values: str = "probabilities"):
        
        """
        :param model: The model object
        :param trends: The computed simulation trends
        :param iteration: The iteration number or the string "last" for plotting final state
        """
        
        self.trends = trends
        self.iteration = iteration
        
        if iteration == 'last':
            self.it = self.trends[-1]['iteration']
            self.ops = self.trends[-1]['status']
        else:
            self.ops = self.trends[iteration]['status'].values()
        
        if values == 'probabilities':
            weights = np.array([el for el in self.ops])
            self.values = p_o * weights + p_p * (1-weights)
            
        elif values == 'weights':
            self.values = np.array([el for el in self.ops])
    
    def get_values(self):
        return self.values
        


In [ ]:
import json
for n_lobbyists in [0, 1, 2, 3]:
    with open(f'../results/{n_lobbyists}_lobbyistsconfig.json', 'r') as f:
        params = json.load(f)
        
        values = ['weights', 'probabilities']

        for value in values:
            for _, (lambda_v, phi_v) in enumerate([(l, p) for l in params['lambda_values'] for p in params['phi_values']]):    
                path = os.path.join(f'../results/{n_lobbyists}_lobbyists/', f'{lambda_v}_{phi_v}/')        
                for run in range(params['nruns']):
                    runpath = os.path.join(path, str(run))
                    with open(runpath+'/status.json', 'r') as f:
                        trends = json.load(f)
                    if not os.path.exists(f'{runpath}/{value}_finaldistribution.png'):
                        print(f'plotting {runpath}')
                        oe = OpinionDistribution(trends, params['p_o'], params['p_p'], values=value)
                        data = oe.get_values()
                        plt.figure(figsize=(10, 6))
                        sns.histplot(data, stat='probability', bins=30)
                        plt.xlim(-0.01, 1.01)
                        plt.ylim(0, 1)
                        plt.xlabel(f"{value.capitalize()} of optimist model")
                        plt.ylabel('% Agents')
                        plt.savefig(f'{runpath}/{value}_finaldistribution.png', dpi=300, bbox_inches='tight', facecolor='white')
                        plt.close()
                    else:
                        continue